<a href="https://colab.research.google.com/github/S2DSLondon/S2DS-Summer24-Polyploy/blob/micon-enrico-rfdiffusion-complete-colab/notebooks/rfdiffusion_pipeline_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1. RFdiffusion + ProteinMPNN + OmegaFold + TM-Metric Full Pipeline

This will run the complete workflow of RFdiffusion pipeline to generate protein design. Please refer to RFdiffusion tutorial jupyter notebook run in google colab for further learning and instructions.  

In [ ]:
# #first command generates key rsa 4096 long. Puts your email at the end of it, save the public key to id_rsa and does all of this with no passphrase being asked
# !ssh-keygen -t rsa -b 4096 -C "enriandri2@gmail.com" -f /root/.ssh/id_rsa -N ""

# #this commands scans GitHub's SSH key and adds it to the known_hosts file, which is used to verify the identity of the remote server. : not sure is necessary
# !ssh-keyscan -t rsa github.com >> ~/.ssh/known_hosts

# #print the private key that you need to copy and paste in github (the key is the last line in the output which starts with "ssh-rsa" and ends with your email included)
# !cat /root/.ssh/id_rsa.pub

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# !mkdir /content/drive/MyDrive/ssh_colab
# !cp /root/.ssh/* /content/drive/MyDrive/ssh_colab
# !ls /content/drive/MyDrive/ssh_colab

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#!mkdir /root/.ssh/
#!cp /content/drive/MyDrive/ssh_colab/* /root/.ssh/

In [ ]:
#%cd /content

#import os

# Check if the directory already exists
#if not os.path.isdir("S2DS-Summer24-Polyploy"):
    # If it does not exist, clone the repository
    !git clone git@github.com:S2DSLondon/S2DS-Summer24-Polyploy.git
    # Change to the repository directory
    # %cd S2DS-Summer24-Polyploy
#else:
    print('S2DS-Summer24-Polyploy already existing')

In [ ]:
#%cd S2DS-Summer24-Polyploy

# Fetch all branches
#!git fetch

# Checkout the specific branch
#!git checkout Enrico_micon_pipeline_v3

# Update your branch in case you had uploaded anything new
#!git pull origin Enrico_micon_pipeline_v3

# List the files to verify you actually managed to go into the branch you want to be
#!ls

#%cd ..
#%pwd


_________________________________________________


_________________________________________________

## A. Set the working directory

In [3]:
# Check current dir
import time
import os
import time
from pathlib import Path

print(os.getcwd())

# Save the current directory path to root_dir variable
root_dir = Path('/content')
root_dir

/content


PosixPath('/content')

## B. Install necessary packages for RFdiffusion

In [4]:
# Install RFdiffusion without installing pytorch ~3minutes
# Install RFdiffusion adapted from Aleksandra Olszewska
# Install RFdiffusion originally by Sergey Ovchinnikov (https://github.com/sokrypton)

tic = time.time()
os.chdir(root_dir)
os.system("git clone https://github.com/RosettaCommons/RFdiffusion.git")  # Original repo

os.system("pip install jedi omegaconf hydra-core icecream pyrsistent pynvml decorator")
os.system("pip install git+https://github.com/NVIDIA/dllogger#egg=dllogger")
# 17Mar2024: adding --no-dependencies to avoid installing nvidia-cuda-* dependencies
os.system("pip install --no-dependencies dgl==2.0.0 -f https://data.dgl.ai/wheels/cu121/repo.html")
os.system("pip install --no-dependencies e3nn==0.3.3 opt_einsum_fx")
os.system("cd RFdiffusion/env/SE3Transformer; pip install .")
os.environ["DGLBACKEND"] = "pytorch"

os.chdir(root_dir / 'RFdiffusion')
!pip install --no-dependencies -e .

# Get PPI Scaffold Examples for RFdiffusion
os.chdir(root_dir / 'RFdiffusion')
!tar -xvf examples/ppi_scaffolds_subset.tar.gz -C examples/

# Download model weights into 'models' dir inside RFdiffusion
os.chdir(root_dir / 'RFdiffusion')
if not os.path.exists('models'):
  os.mkdir('models')
os.chdir('./models')
!wget http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt
!wget http://files.ipd.uw.edu/pub/RFdiffusion/e29311f6f1bf1af907f9ef9f44b8328b/Complex_base_ckpt.pt
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/60f09a193fb5e5ccdc4980417708dbab/Complex_Fold_base_ckpt.pt
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/74f51cfb8b440f50d70878e05361d8f0/InpaintSeq_ckpt.pt
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/76d00716416567174cdb7ca96e208296/InpaintSeq_Fold_ckpt.pt
!wget http://files.ipd.uw.edu/pub/RFdiffusion/5532d2e1f3a4738decd58b19d633b3c3/ActiveSite_ckpt.pt
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/12fc204edeae5b57713c5ad7dcb97d39/Base_epoch8_ckpt.pt

#!wget http://files.ipd.uw.edu/pub/RFdiffusion/f572d396fae9206628714fb2ce00f72e/Complex_beta_ckpt.pt

# original structure prediction weights
!wget http://files.ipd.uw.edu/pub/RFdiffusion/1befcb9b28e2f778f53d47f18b7597fa/RF_structure_prediction_weights.pt

toc = time.time()
print("It took {:.2f} minutes to run this script".format((toc - tic)/60))

Obtaining file:///content/RFdiffusion
  Preparing metadata (setup.py) ... done
  Running setup.py develop for rfdiffusion
ppi_scaffolds/
ppi_scaffolds/HEEHE_2518_ss.pt
ppi_scaffolds/HHH_b1_00611_ss.pt
ppi_scaffolds/HHH_b1_00312_ss.pt
ppi_scaffolds/01d461f506ab04f7d0ebca3e58138a95_0001_ss.pt
ppi_scaffolds/HHH_b2_00055_adj.pt
ppi_scaffolds/HHH_b1_06097_adj.pt
ppi_scaffolds/HHH_b2_03550_adj.pt
ppi_scaffolds/HHH_b1_01422_ss.pt
ppi_scaffolds/HHH_b2_02422_ss.pt
ppi_scaffolds/HHH_b2_02121_ss.pt
ppi_scaffolds/HHH_b2_04798_adj.pt
ppi_scaffolds/HHH_b1_01645_adj.pt
ppi_scaffolds/915c467777ae14ad0cbad57e3ac0e165_0001_adj.pt
ppi_scaffolds/HHH_b2_07193_adj.pt
ppi_scaffolds/aadf5aa4c991a2bbf69164fcaa2c5d6f_0001_1_ss.pt
ppi_scaffolds/HHH_b1_04925_ss.pt
ppi_scaffolds/HHH_b1_03924_adj.pt
ppi_scaffolds/0cd074c2d6d3de76f1399c5a775575d2_0001_1_adj.pt
ppi_scaffolds/HHH_b1_03663_ss.pt
ppi_scaffolds/HHH_eva_0296_adj.pt
ppi_scaffolds/HHH_b2_03436_ss.pt
ppi_scaffolds/HHH_b2_03817_ss.pt
ppi_scaffolds/HHH_b2_0411

## C. Clone ProteinMPNN

In [5]:
# Clone protein MPNN ~0minutes
tic = time.time()

os.chdir(root_dir)
!git clone https://github.com/dauparas/ProteinMPNN.git

toc = time.time()
print("It took {:.2f} minutes to run this script".format((toc - tic)/60))

Cloning into 'ProteinMPNN'...
remote: Enumerating objects: 634, done.
remote: Counting objects: 100% (218/218), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 634 (delta 174), reused 146 (delta 146), pack-reused 416
Receiving objects: 100% (634/634), 119.89 MiB | 24.34 MiB/s, done.
Resolving deltas: 100% (285/285), done.
It took 0.11 minutes to run this script


## D. Clone OmegaFold

In [6]:
# Clone OmegaFold ~0.17 minutes
tic = time.time()

os.chdir(root_dir)
!git clone https://github.com/HeliXonProtein/OmegaFold.git
!pip install biopython

os.chdir(root_dir / 'OmegaFold')
!pip install --no-dependencies -e .

toc = time.time()
print("It took {:.2f} minutes to run this script".format((toc - tic)/60))

Cloning into 'OmegaFold'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (25/25), done.
Receiving objects: 100% (208/208), 726.66 KiB | 16.51 MiB/s, done.
Resolving deltas: 100% (125/125), done.
remote: Total 208 (delta 50), reused 38 (delta 38), pack-reused 145
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 31.1 MB/s eta 0:00:00
Obtaining file:///content/OmegaFold
  Preparing metadata (setup.py) ... done
  Running setup.py develop for OmegaFold
It took 0.12 minutes to run this script


## E. Install TM-Align for metrics

In [11]:
# Step 1: Install necessary tools
!pip install biopython
!apt-get update
!apt-get install -y build-essential

# Step 2: Download the TM-align source code
!wget https://zhanglab.ccmb.med.umich.edu/TM-align/TMalign.cpp -O TMalign.cpp

# Step 3: Compile the source code
!g++ -O3 -ffast-math -lm -o TMalign TMalign.cpp

# Verify the installation
!./TMalign -h

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,129 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [2,721 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,073 kB]
Ign:13 https://r2u.stat.illinois.edu/ubuntu jammy 

________________________________________________________________________________________________________________________________________________________________________________________

# Part 2. Complete Workflow of RFdiffusion pipeline
________________________________________________________________________________________________________________________________________________________________________________________

## A. RFdiffusion Pipeline Functions

In [7]:
# Check and Setup work dir #cd /home/ubuntu/output
# import packages
import glob
import os
import re
import requests
import subprocess
import time


def setup_folder(root_dir):

    # Define the base folder in AWS
    base_folder = f'{root_dir}/work_flow'

    # Define the subfolders
    subfolders = ['RFdiffusion_output', 'mpnn_output', 'omegafold_output', 'native_proteins']

    # Check if the base folder already exists
    if not os.path.exists(base_folder):
        # Create the base folder
        os.makedirs(base_folder, exist_ok=True)

        # Create the subfolders
        for subfolder in subfolders:
            os.makedirs(os.path.join(base_folder, subfolder), exist_ok=True)

        print("Folder structure created successfully.")
    else:
        print("The folder 'work_flow' already exists.")


def run_rfdiffusion(input_file: str, output_dir_and_prefix: str, number_proteins: int,
                    residues: str = None, guide_scale: int = None,
                    substrate_name: str = None, model_weights: str = None,
                    contig_length: str = None, guiding_potentials: str = None):
    """
    If the name of the substrate is specified (e.g. LLK), it must be present in the input pdb file.
    """
    if input_file is not None and not os.path.exists(input_file):
      raise ValueError("Input file does not exist")

    # Input checks
    if residues is None and guide_scale is not None:
        raise ValueError("Please fill in residues")

    # Create function call
    sh_call = "./RFdiffusion/scripts/run_inference.py"
    #sh_call = "run_inference.py"
    sh_call += " inference.output_prefix=" + output_dir_and_prefix
    sh_call += " inference.num_designs=" + str(number_proteins)

    # Optional arguments
    if input_file != "":
        sh_call += " inference.input_pdb=" + input_file

    if residues is not None:
        # contigmap.contigs line must be in single quotes
        sh_call += " 'contigmap.contigs=" + residues + "'"

    if contig_length is not None:
        sh_call += " contigmap.length=" + contig_length

    if guide_scale is not None:
        sh_call += " potentials.guide_scale=" + str(guide_scale)

    if guiding_potentials is not None:
        # potentials.guiding_potentials line must be in single quotes
        # the guiding_potentials must be in double quotes
        # sh_call += " " + repr('potentials.guiding_potentials=["type:substrate_contacts,s:1,r_0:8,rep_r_0:5.0,rep_s:2,rep_r_min:1"]')
        sh_call += " 'potentials.guiding_potentials=[" + '"' + guiding_potentials + '"' + "]'"

    if substrate_name is not None:
        sh_call += " potentials.substrate=" + substrate_name

    if model_weights is not None:
        sh_call += " inference.ckpt_override_path=" + model_weights

    print("Running RFdiffusion with call", sh_call)

    os.system(sh_call)


def run_protein_mpnn(input_file: str, output_dir: str,
                    num_seq_per_target: int = 10, #default
                    sampling_temp: str = "0.1", #default
                    seed: int = 0 , #default
                    batch_size: int = 1, #default
                    model_name: str = "v_48_020"): #default

    # Check if input file exists
    if not os.path.exists(input_file):
        raise ValueError("input_file does not exist")

    sh_call= "python ./ProteinMPNN/protein_mpnn_run.py"
    sh_call+= " --pdb_path " + input_file
    sh_call+= " --out_folder " + output_dir
    sh_call+= " --num_seq_per_target " + str(num_seq_per_target)
    # Must be a string, can contain multiple temps, e.g.  "0.1 0.2"
    sh_call+= " --sampling_temp " + '"' + sampling_temp + '"'
    sh_call+= " --seed " + str(seed)
    sh_call+= " --batch_size " + str(batch_size)
    sh_call+= " --model_name " + model_name

    print("Running ProteinMPNN with call", sh_call)

    os.system(sh_call)


def run_omegafold(input_file: str, output_file: str):

    if not os.path.exists(input_file):
        raise ValueError("input_file does not exist")

    sh_call= "omegafold"
    sh_call+= " " + input_file
    sh_call+= " " + output_file


    print("Running omegafold with call", sh_call)

    os.system(sh_call)


def download_pdb(pdb_code, target_directory='./work_flow/native_proteins'):
    # Ensure the target directory exists
    os.makedirs(target_directory, exist_ok=True)

    # Construct the full path for the PDB file
    pdb_file_path = os.path.join(target_directory, f"{pdb_code.upper()}.pdb")

    # URL for downloading the PDB file
    url = f"https://files.rcsb.org/download/{pdb_code.upper()}.pdb"

    # Request the PDB file
    response = requests.get(url)

    if response.status_code == 200:
        # Write the content to the PDB file
        with open(pdb_file_path, 'wb') as f:
            f.write(response.content)
        print(f"PDB file {pdb_code.upper()} downloaded successfully to {pdb_file_path}")
    else:
        print(f"Failed to download PDB file {pdb_code.upper()}. Status code: {response.status_code}")


def extract_scores(ref_protein, generated_protein): #it takes as inputs the path of the two pdb files
    ref_protein_abs = os.path.abspath(ref_protein)
    generated_protein_abs = os.path.abspath(generated_protein)

    # Run the TMalign command
    tmalign_command = ["./TMalign", ref_protein_abs, generated_protein_abs]
    result = subprocess.run(tmalign_command, capture_output=True, text=True)

    if result.returncode == 0:
        output = result.stdout

        # Extract TM-scores from the output using regex
        tm_scores = re.findall(r"TM-score=\s+([\d.]+)", output)
        if len(tm_scores) >= 2:
            tm_score_1 = float(tm_scores[0])
            tm_score_2 = float(tm_scores[1])
        else:
            print("TM-scores not found in the output.")
            return None

        # Extract RMSD score from the output using regex
        rmsd_score_match = re.findall(r"RMSD=\s+([\d.]+)", output)
        if rmsd_score_match:
            rmsd_score = float(rmsd_score_match[0])
        else:
            print("RMSD score not found in the output.")
            return None

        return tm_score_1, tm_score_2, rmsd_score
    else:
        print(f"Error running TMalign for {generated_protein}. Return code: {result.returncode}")
        print(result.stderr)
        return None

#we will need this function to isolate the correct pdb file from the files that omegafold will give as output
def find_score_file(pdb_code):
    directory = f'./work_flow/omegafold_output/{pdb_code}'
    score_file_prefix = f'{pdb_code}_score'

    for filename in os.listdir(directory):
        if filename.startswith(score_file_prefix):
            return os.path.join(directory, filename)

    return None


# Main pipeline (RFdiffusion using PDB code + ProteinMPNN + OmegaFold)
def process_protein(pdb_code, residues_input):

    tic = time.time()

    download_pdb(pdb_code)
    # Ensure the pdb_code is in uppercase
    pdb_code_upper = pdb_code.upper()

    # Construct the input and output paths
    RF_input = f"./work_flow/native_proteins/{pdb_code_upper}.pdb"
    RF_output = f"./work_flow/RFdiffusion_output/{pdb_code}/{pdb_code}_scaffold"

    # Check if the input file exists
    if not os.path.exists(RF_input):
        raise ValueError(f"Input file {RF_input} does not exist")

    # Run RFdiffusion
    run_rfdiffusion(
        input_file=RF_input,
        output_dir_and_prefix=RF_output,
        number_proteins=1,
        residues=residues_input
    )

    # Run ProteinMPNN
    run_protein_mpnn(
        input_file=f"./work_flow/RFdiffusion_output/{pdb_code}/{pdb_code}_scaffold_0.pdb",
        output_dir=f"./work_flow/mpnn_output/{pdb_code}",
        num_seq_per_target=1,
        sampling_temp="0.1",
        seed=0,
        batch_size=1,
        model_name="v_48_020"
    )

    # Run OmegaFold
    run_omegafold(
        input_file=f"./work_flow/mpnn_output/{pdb_code}/seqs/{pdb_code}_scaffold_0.fa",
        output_file=f"./work_flow/omegafold_output/{pdb_code}"
    )

    #now we change the names in the omegafold folder such that in the name there is information about the score:
    #we perform this task in 4 steps
    # 1. Define the folder containing the .pdb files
    folder_path = f'./work_flow/omegafold_output/{pdb_code}'

    # 2. List all files in the folder
    files = os.listdir(folder_path)

    # 3. Define a regular expression to extract the relevant parts of the filename
    pattern = re.compile(r"([a-zA-Z0-9]+)_.*score=([\d.]+)")

    # 4. Process each file
    for filename in files:
       if filename.endswith('.pdb'):
           match = pattern.search(filename)
           if match:
                alphanumeric_part = match.group(1)
                score_part = match.group(2)
                new_filename = f"{alphanumeric_part}_score_{score_part}.pdb"
                old_file = os.path.join(folder_path, filename)
                new_file = os.path.join(folder_path, new_filename)
                os.rename(old_file, new_file)
                print(f"Renamed '{filename}' to '{new_filename}'")


    #now let's print the metrics between the output of the first and third steps of the pipeline
    output_first_step=f'./work_flow/RFdiffusion_output/{pdb_code}/{pdb_code}_scaffold_0.pdb'
    output_third_step=find_score_file(pdb_code)

    toc = time.time()
    print("It took {:.2f} minutes to run RFdiffusion + ProteinMPNN + OmegaFold".format((toc - tic)/60))

    return extract_scores(output_first_step, output_third_step)

# TM and LM score metric to determine quality of newly designed protein and compare it from native protein
def visual_comparison(ref_protein, generated_protein): #it takes as inputs the path of the two pdb files
    ref_protein_abs = os.path.abspath(ref_protein)
    generated_protein_abs = os.path.abspath(generated_protein)

    # Run the TMalign command
    tmalign_command = ["./TMalign", ref_protein_abs, generated_protein_abs]
    result = subprocess.run(tmalign_command, capture_output=True, text=True)

    if result.returncode == 0:
      output = result.stdout
      print(output)
    else:
      print(f"Error running TMalign for {generated_protein}. Return code: {result.returncode}")
      print(result.stderr)





## B. RFdiffusion run

In [12]:
#import os
#from rfdiffusion_pipeline import process_protein, setup_folder, visual_comparison, find_score_file


## NB, important to set current dir as root_dir as this is where RFdif etc were installed
## root_dir must be the same that was used above when installing packages
root_dir = Path('/content')
os.chdir(root_dir)
print(root_dir)

setup_folder(root_dir)

# user must change below: (1) PDB code & (2) amino acid positions
result=process_protein('7SH6', "[20-30/157-163/20-40/157-163/20-30]")
print(result)

# user must change PDB code below to match the PDB code above
pdb_code_native='7SH6'
pdb_native=f"./work_flow/native_proteins/{pdb_code_native.upper()}.pdb"
pdb_artificial=find_score_file(pdb_code_native)
print(pdb_native)
print(pdb_artificial)
visual_comparison(pdb_native,pdb_artificial)

/content
The folder 'work_flow' already exists.
PDB file 7SH6 downloaded successfully to ./work_flow/native_proteins/7SH6.pdb
Running RFdiffusion with call ./RFdiffusion/scripts/run_inference.py inference.output_prefix=./work_flow/RFdiffusion_output/7SH6/7SH6_scaffold inference.num_designs=1 inference.input_pdb=./work_flow/native_proteins/7SH6.pdb 'contigmap.contigs=[20-30/157-163/20-40/157-163/20-30]'
Running ProteinMPNN with call python ./ProteinMPNN/protein_mpnn_run.py --pdb_path ./work_flow/RFdiffusion_output/7SH6/7SH6_scaffold_0.pdb --out_folder ./work_flow/mpnn_output/7SH6 --num_seq_per_target 1 --sampling_temp "0.1" --seed 0 --batch_size 1 --model_name v_48_020
Running omegafold with call omegafold ./work_flow/mpnn_output/7SH6/seqs/7SH6_scaffold_0.fa ./work_flow/omegafold_output/7SH6
Renamed '7SH6_scaffold_0, score=2.3832, global_score=2.3832, fixed_chains=[], designed_chains=['A'], model_name=v_48_020, git_hash=8907e6671bfbfc92303b5f79c4b5e6ce47cdef57, seed=142.pdb' to '7SH6_sc

________________________________________________________________________________________________________________________________________________________________________________________

    This is the end of the script. Please proceed to further validation using either

    1) PYMOL or similar software with a commercial license or
    
    2) by following the procedure described in the google colab tutorial (splitting the artificial proteins into smaller pieces and compare each one of these with the active site of the native protein).
    
    Have a great day! :)
________________________________________________________________________________________________________________________________________________________________________________________